In [ ]:
# Install deepinv (skip if already installed)
%pip install deepinv

<!-- MathJax macro definitions inserted automatically -->
$$
\newcommand{\forw}[1]{{A\left({#1}\right)}}
\newcommand{\noise}[1]{{N\left({#1}\right)}}
\newcommand{\inverse}[1]{{R\left({#1}\right)}}
\newcommand{\inversef}[2]{{R\left({#1},{#2}\right)}}
\newcommand{\inversename}{R}
\newcommand{\reg}[1]{{g_\sigma\left({#1}\right)}}
\newcommand{\regname}{g_\sigma}
\newcommand{\sensor}[1]{{\eta\left({#1}\right)}}
\newcommand{\datafid}[2]{{f\left({#1},{#2}\right)}}
\newcommand{\datafidname}{f}
\newcommand{\distance}[2]{{d\left({#1},{#2}\right)}}
\newcommand{\distancename}{d}
\newcommand{\denoiser}[2]{{\operatorname{D}_{{#2}}\left({#1}\right)}}
\newcommand{\denoisername}{\operatorname{D}_{\sigma}}
\newcommand{\xset}{\mathcal{X}}
\newcommand{\yset}{\mathcal{Y}}
\newcommand{\group}{\mathcal{G}}
\newcommand{\metric}[2]{{d\left({#1},{#2}\right)}}
\newcommand{\loss}[1]{{\mathcal\left({#1}\right)}}
\newcommand{\conj}[1]{{\overline{#1}^{\top}}}
$$


# Use a pretrained model

Follow this example to reconstruct images using a pretrained model in one line.

We show three sets of general pretrained reconstruction methods, including:

* Pretrained feedforward [Reconstruct Anything Model (RAM)](https://deepinv.github.io/deepinv/api/stubs/deepinv.models.RAM.html);
* Plug-and-play with a pretrained denoiser.
* Pretrained diffusion model;

See pretrained models for a principled comparison between methods demonstrated in this example.

> **Tip**
>
>
> * Want to use your own dataset? See [`basics/demo_custom_dataset.py`](https://deepinv.github.io/deepinv/auto_examples/basics/demo_custom_dataset.html#sphx-glr-auto-examples-basics-demo-custom-dataset-py)
> * Want to use your own physics? See [`basics/demo_custom_physics.py`](https://deepinv.github.io/deepinv/auto_examples/basics/demo_custom_physics.html#sphx-glr-auto-examples-basics-demo-custom-physics-py)


In [ ]:
import deepinv as dinv
import torch

device = dinv.utils.get_freer_gpu() if torch.cuda.is_available() else "cpu"

Let's say you want to reconstruct a butterfly from noisy, blurry measurements:


In [ ]:
# Ground truth
x = dinv.utils.load_example("butterfly.png", device=device)

# Define physics
physics = dinv.physics.BlurFFT(
    x.shape[1:],
    filter=dinv.physics.blur.gaussian_blur((5, 5)),
    noise_model=dinv.physics.GaussianNoise(
        sigma=0.1, rng=torch.Generator(device=x.device).manual_seed(123)
    ),
    device=device,
)

y = physics(x)

For each model, define model in one line and reconstruct in one line.
Pretrained Reconstruct Anything Model:

> **Seealso**
>
> See [`models/demo_foundation_model.py`](https://deepinv.github.io/deepinv/auto_examples/models/demo_foundation_model.html#sphx-glr-auto-examples-models-demo-foundation-model-py) for further one-line examples for the RAM model across various domains.
>


In [ ]:
model = dinv.models.RAM(pretrained=True, device=device)

with torch.no_grad():
    x_hat1 = model(y, physics)

PnP algorithm with pretrained denoiser:

> **Seealso**
>
> See pretrained denoisers for a full list of denoisers that can be plugged into iterative/sampling algorithms.
>
> See [`plug-and-play/demo_PnP_DPIR_deblur.py`](https://deepinv.github.io/deepinv/auto_examples/plug-and-play/demo_PnP_DPIR_deblur.html#sphx-glr-auto-examples-plug-and-play-demo-PnP-DPIR-deblur-py) for a further example of using plug-and-play with a pretrained denoiser.
>


In [ ]:
denoiser = dinv.models.DRUNet(device=device)
model = dinv.optim.DPIR(sigma=0.1, denoiser=denoiser, device=device)

x_hat2 = model(y, physics)

Pretrained diffusion model (we reduce the image size for demo speed on CPU, as diffusion model is slow):

> **Seealso**
>
> See [`sampling/demo_ddrm.py`](https://deepinv.github.io/deepinv/auto_examples/sampling/demo_ddrm.html#sphx-glr-auto-examples-sampling-demo-ddrm-py) for a further example of using a pretrained diffusion model.
>


In [ ]:
model = dinv.sampling.DDRM(denoiser, sigmas=torch.linspace(1, 0, 20)).to(device)

x_hat3 = model(y, physics)

Plot results


In [ ]:
dinv.utils.plot(
    {
        "Ground Truth": x,
        "Measurement": y,
        "Pretrained \nRAM": x_hat1,
        "Pretrained \nPnP": x_hat2,
        "Pretrained \nDiffusion": x_hat3,
    },
    subtitles=[
        "PSNR:",
        f"{dinv.metric.PSNR()(y, x).item():.2f} dB",
        f"{dinv.metric.PSNR()(x_hat1, x).item():.2f} dB",
        f"{dinv.metric.PSNR()(x_hat2, x).item():.2f} dB",
        f"{dinv.metric.PSNR()(x_hat3, x).item():.2f} dB",
    ],
    figsize=(10, 5),
)


# sphinx_gallery_start_ignore
assert dinv.metric.PSNR()(x_hat1, x).item() > 19
# sphinx_gallery_end_ignore

🎉 Well done, you now know how to use pretrained models!

## What's next?
* Check out the [example on how to fine-tune a foundation model to your own problem](https://deepinv.github.io/deepinv/auto_examples/models/demo_foundation_model.html#sphx-glr-auto-examples-models-demo-foundation-model-py).
* See pretrained models for a comparison between methods demonstrated in this example.
* See diffusion and iterative for how to fully customize your sampling or iterative algorithm using a pretrained denoiser.
